### **rnnを実装してみる**

## **必要な関数・ライブラリ等の準備**

In [1]:
# データ加工・処理・分析モジュール
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random

%matplotlib inline

import time

In [2]:
def set_time(dataframe, col_name):
    '''
    to_datetimeを使うための前処理
    '''
    dataframe[col_name] = dataframe[col_name].map(lambda x : transform_time(x))
    return dataframe

In [3]:
def transform_time(x):
    '''
    set_time内で使う関数
    to_datetimeで24時をサポートしないので00に変更する処理
    '''
    str_x = str(x)
    res = ''
    if str(x)[8:10] == '24':
        res = str_x[0:4] + '-' + str_x[4:6] + '-' + str_x[6:8] + ' 00:'+str_x[10:12] 
    else:
        res = str_x[0:4] + '-' + str_x[4:6] + '-' + str_x[6:8] + ' '+ str_x[8:10] +':'+str_x[10:12]
    return res

## **データの準備**

In [4]:
# データの読み込み
# 処理済みデータを読み込む。無かったら作る
try:
    output_data = pd.read_csv('data/processed_data/out_put.tsv', delimiter = '\t')
    output_data['datetime'] = output_data['datetime'].map(lambda x : pd.to_datetime(x))
    
except:
    # train_kwhをエクセル等で開くとdatetimeが指数表示に直される可能性がある
    # その場合うまくいかないので201201010120の形になってることを確認する必要あり
    output_data = pd.read_csv('data/raw_data/train_kwh.tsv', delimiter = '\t')

    # datetimeの行をpd.Timestampのインスタンスに変更
    output_data = set_time(output_data, 'datetime')
    output_data['datetime'] = output_data['datetime'].map(lambda x : pd.to_datetime(x))

    # 30分ごとに合計を集計
    output_data = output_data.set_index('datetime').groupby(pd.TimeGrouper(freq='1800s', closed='left')).sum()

    output_data.to_csv('data/processed_data/out_put.tsv', sep='\t') 
    
output_data.head()

,datetime,SOLA01,SOLA02,SOLA03
0,2012-01-01 00:00:00,0,0.0,NaN
1,2012-01-01 00:30:00,0,0.0,NaN
2,2012-01-01 01:00:00,0,0.0,NaN
3,2012-01-01 01:30:00,0,0.0,NaN
4,2012-01-01 02:00:00,0,0.0,NaN


In [5]:
# データの読み込み
# 処理済みデータを読み込む。無かったら作る
try:
    amd_data = pd.read_csv('data/processed_data/amd_data.tsv', delimiter='\t')
    amd_data['datetime'] = amd_data['datetime'].map(lambda x : pd.to_datetime(x))
    
except:
    # 河口湖アメダスのデータを使って予測する, amd_49251
    # 各amdidはamd_masterに記載されている
    amd_data = pd.read_csv('data/raw_data/amd_49251.tsv', delimiter = '\t')

    amd_data = set_time(amd_data, 'datetime')
    amd_data['datetime'] = amd_data['datetime'].map(lambda x : pd.to_datetime(x))

    # 30分ごとに平均を集計
    amd_data = amd_data.set_index('datetime').groupby(pd.TimeGrouper(freq='1800s', closed='left')).mean()

    amd_data.to_csv('data/processed_data/amd_data.tsv', sep='\t')

amd_data.head()

,datetime,pr,f_pr,max_ws,f_max_ws,ave_wv,f_ave_wv,ave_ws,f_ave_ws,max_tp,f_max_tp,min_tp,f_min_tp,sl,f_sl,sd,f_sd,dsd,f_dsd
0,2012-01-01 00:00:00,0.0,0.0,22.333333,0.0,20.333333,0.0,10.000000,0.0,-15.666667,0.0,-26.333333,0.0,0.0,2.0,0.0,0.0,0.0,0.0
1,2012-01-01 00:30:00,0.0,0.0,15.333333,0.0,25.000000,0.0,9.333333,0.0,-34.666667,0.0,-40.000000,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,2012-01-01 01:00:00,0.0,0.0,20.000000,0.0,25.000000,0.0,10.666667,0.0,-35.333333,0.0,-43.666667,0.0,0.0,2.0,0.0,0.0,0.0,0.0
3,2012-01-01 01:30:00,0.0,0.0,20.333333,0.0,21.666667,0.0,11.333333,0.0,-36.666667,0.0,-42.333333,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,2012-01-01 02:00:00,0.0,0.0,25.333333,0.0,21.000000,0.0,13.333333,0.0,-40.666667,0.0,-48.000000,0.0,0.0,2.0,0.0,0.0,0.0,0.0


In [6]:
# モデル構築のためにデータを分割する

# 日射量の欠損値を一つ前の値で置換/output_dataに関してはSOLA01はnullなし
amd_data['sl'] = amd_data['sl'].fillna(method='bfill')

# 学習に必要なデータ
# 2012/01/01 00:00 ~ 2015/12/30 20:00のamdデータを用いて
# 2012/01/03 03:30 ~ 2015/12/31 23:30のデータを予測する
train_x_startID = amd_data[amd_data['datetime'] == pd.to_datetime('2012-01-01 00:00')].index[0]
train_x_endID = amd_data[amd_data['datetime'] == pd.to_datetime('2015-12-30 20:00')].index[0]
train_y_startID = amd_data[amd_data['datetime'] == pd.to_datetime('2012-01-03 03:30')].index[0]
train_y_endID = amd_data[amd_data['datetime'] == pd.to_datetime('2015-12-31 23:30')].index[0]

train_amd_data = list(amd_data['sl'][train_x_startID:(train_x_endID+1)])
train_output_data = np.array(output_data['SOLA01'][train_y_startID:(train_y_endID+1)])

#時系列データのリストにする
input_list = []
for i in range(0, len(train_amd_data) - 48):
    input_list.append(train_amd_data[i : i + 49])
input_list = np.array(input_list)

# 予測に必要なデータ
# 2015/12/30 20:30 ~ 2017/3/30 20:00のamdデータを用いて
# 2016/01/01 00:00 ~ 2017/3/31 23:30のoutputデータを予測する
test_startID = amd_data[amd_data['datetime'] == pd.to_datetime('2015-12-30 20:30')].index[0]
test_endID = amd_data[amd_data['datetime'] == pd.to_datetime('2017-3-30 20:00')].index[0]

test_amd_data = list(amd_data['sl'][test_startID:(test_endID+1)])

In [7]:
X = np.array(input_list).reshape(len(input_list), 49, 1)
Y = np.array(train_output_data).reshape(len(input_list), 1 )

N_train = int(len(input_list) * 0.9)
N_val = len(input_list) - N_train
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=N_val)

## **モデルの構築**

前日の20時までの河口個のアメダスの日射量データを用いて翌日の00:00~23:30を予測する
(例)

8/17の00:00 ~ 23:30を予測するとき

8/17の23:30を8/15 20:00 ~ 8/16 20:00

8/17の23:00を8/15 19:30 ~ 8/16 19:30

....

8/17の00:00を8/14 20:30 ~ 8/15 20:30で予測する

参考サイト：http://www.madopro.net/entry/char_level_lm_with_simple_rnn

　　　　　　https://qiita.com/yukiB/items/f6314d2861fc8d9b739f

          https://www.tensorflow.org/install/migration
          

In [14]:
class simpleLSTM:
    def __init__(self, X, Y):
        self.input_layer_size = 1 #入力層の数、一つ一つのデータはslの値ひとつだけなので1
        self.hidden_layer_size = 47 # 隠れ層の数、とりあえず系列データの長さ分だけ用意しておく
        self.output_layer_size = 1 #出力層の数、求める値は時間あたりの発電量の値1つなので1
        self.batch_size = 100 #バッチサイズ、適当
        self.chunk_size = 49 # 一回の系列データの長さ
        self.learning_rate = 0.01 # 学習率適当
        self.forget_bias = 0.8  # 忘却率
        self.X = X # 入力
        self.Y = Y # 教師
        self.epochs = 1000 #エポック数
        
    def get_batch(self):
        rnum = [random.randint(0, len(self.X) - 1) for x in range(self.batch_size)]
        xs = np.array([[y for y in list(self.X[r])] for r in rnum])
        ys = np.array([self.Y[r] for r in rnum])
        return xs, ys
        
    def inference(self, input_ph, inistate_ph):
        
        # 重みとバイアスの初期化
        hidden_w = tf.Variable(tf.truncated_normal([self.input_layer_size, self.hidden_layer_size], stddev=0.01), name='hidden_w')
        hidden_b = tf.Variable(tf.truncated_normal([self.hidden_layer_size]), name='hidden_b')
        output_w = tf.Variable(tf.truncated_normal([self.hidden_layer_size, self.output_layer_size], stddev=0.01), name='output_w')
        output_b = tf.Variable(tf.truncated_normal([self.output_layer_size]), name='output_b')
        
        #　input_phの整型  
        in1 = tf.transpose(input_ph, [1, 0, 2]) 
        in2 = tf.reshape(in1, [-1, self.input_layer_size]) 
        in3 = tf.matmul(in2, hidden_w) + hidden_b
        in4 = tf.split(in3, self.chunk_size, 0)
          
            
        # BasicLSTMCellを定義
        cell = tf.contrib.rnn.BasicLSTMCell(self.hidden_layer_size, forget_bias=self.forget_bias, state_is_tuple=False)
        rnn_outputs, states = tf.contrib.rnn.static_rnn(cell, in4, initial_state=inistate_ph)
        output = tf.matmul(rnn_outputs[-1], output_w) + output_b
        results = [hidden_w, output_w, hidden_w, hidden_b]
        return output, states, results
        
    def loss(self, output_ph, actual_ph):
        cost = tf.reduce_mean(tf.abs(output_ph - actual_ph))
        tf.summary.scalar('loss', cost)
        return cost
    
    def training(self, cost):
        with tf.name_scope("training") as scope:
            optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate, beta1=0.9, beta2=0.999).minimize(cost)
            return optimizer
    
    def train(self):

        random.seed(0)
        np.random.seed(0)
        tf.set_random_seed(0)
        
        with tf.Graph().as_default():
            # 変数の用意
            input_ph = tf.placeholder('float', [None, self.chunk_size, self.input_layer_size], name='input')
            actual_ph = tf.placeholder('float', [None, self.output_layer_size], 'actual_value')
            inistate_ph = tf.placeholder('float', [None, self.hidden_layer_size * 2], 'inistate')

            prediction, state, weights = self.inference(input_ph, inistate_ph)
            cost = self.loss(prediction, actual_ph)
            optimizer = self.training(cost)

            # TensorBoardで可視化する
            summary = tf.summary.merge_all()
            # 初期化
            init = tf.global_variables_initializer()
                
            # ここから学習
            with tf.Session() as sess:
                # 学習したモデルも保存しておく
                saver = tf.train.Saver()
                summary_writer = tf.summary.FileWriter("/tmp/tensorflow_log", graph=sess.graph)
                sess.run(init)

                for epoch in range(self.epochs):
                    inputs, actuals = self.get_batch()
                    train_dict = {
                        input_ph:      inputs,
                        actual_ph:     actuals,
                        inistate_ph:     np.zeros((self.batch_size, self.hidden_layer_size * 2)),
                    }
                    
                    sess.run(optimizer, feed_dict=train_dict)

                    if (epoch) % 100 == 0:
                        summary_str, train_loss = sess.run([summary, cost], feed_dict=train_dict)
                        print("train#%d, train loss: %e" % (epoch, train_loss))
                        summary_writer.add_summary(summary_str, epoch)
                        
                datas = sess.run(weights)
                saver.save(sess, "./data/model/wiki2.ckpt")

In [15]:
model_01 = simpleLSTM(X, Y)

In [16]:
model_01.train()

train#0, train loss: 5.476757e+02
train#100, train loss: 8.313314e+02
train#200, train loss: 6.303976e+02
train#300, train loss: 5.922646e+02
train#400, train loss: 5.870091e+02
train#500, train loss: 5.954622e+02
train#600, train loss: 4.978170e+02
train#700, train loss: 7.186433e+02
train#800, train loss: 6.805502e+02
train#900, train loss: 5.057543e+02
